<a href="https://colab.research.google.com/github/siddhesh1594/siddhesh1594.github.io/blob/main/linesearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.typing import NDArray

class rosenbrock:
    def __init__(self,a=1.0,b=100.0):
        self.a: float = a
        self.b: float = b

    def objective(self,x: NDArray) -> float:
        x1, x2 = x[0], x[1]
        arr = (self.a - x1)**2 + self.b*(x2 - x1**2)**2
        return float(arr) # chatgpt says it should return a scalar value and not np.array([arr])
        # but with scalar value the loop got stuck
    def grad_objective(self,x: NDArray) -> NDArray:
        x1, x2 = x[0], x[1]
        dfdx1 = -2*(self.a - x1) - 4*self.b*x1*(x2 - x1**2)
        dfdx2 =  2*self.b*(x2 - x1**2)
        return np.array([dfdx1, dfdx2])

def steepest_descent(
        x:NDArray,
        problem:rosenbrock,
        tol:float = 1e-6,
        iteration_limit:int = 100000
        ):
    iterations = 0
    print("\n---Steepest Descen Logs---")
    'rosenbrock.grad_objective() missing 1 required positional argument: x'
    '# call object and use the function # error message'

    while iterations < iteration_limit:
        'function = rosenbrock() # create object # no need of this line as problem is already calling rosenborck'

        #compute gradient at current x
        grad = problem.grad_objective(x)

        #search direction
        direction = -grad

        grad_norm = np.linalg.norm(direction,ord=6)

        #log iteration and gradient norm
        #print(f"Iter {iterations:4d} | x = {x} | f(x) = {problem.objective(x):.6e} | ||grad|| = {grad_norm:.6e}")
        #convergence check
        if grad_norm <= tol:
            print("\nConverged Successfully.")
            return x,iterations,problem.objective(x)
        # include line search inside steepest descent
        alpha = linesearch(base_x=x,direction=direction)
        #update x
        x = x + alpha * direction
        iterations += 1

    print("\nreached iteration limit without convergence")
    return x,iterations,problem.objective(x)

def linesearch(
    base_x: NDArray,
    direction: NDArray,
    alpha_min: float = 1e-6,
    alpha_max: float = 1.0,
    alpha_ini: float = 0.5e-2,
    m1: float = 0.01,
    m2: float = 0.90,
    max_iter: int = 100,
) -> float:
    fn = rosenbrock()

    # Precompute at base point
    f0 = fn.objective(base_x)          # scalar
    g0 = fn.grad_objective(base_x)     # (n,)
    slope0 = g0 @ direction            # scalar, should be < 0 for descent

    if slope0 >= 0:
        raise ValueError("Direction is not a descent direction!")

    alpha_L = alpha_min
    alpha_R = alpha_max
    alpha = alpha_ini

    for k in range(max_iter):
        x_trial = base_x + alpha * direction
        phi = fn.objective(x_trial)
        g_trial = fn.grad_objective(x_trial)
        phi_prime = g_trial @ direction

        armijo_rhs = f0 + m1 * alpha * slope0

        armijo_ok = (phi <= armijo_rhs)
        curvature_ok = (abs(phi_prime) <= m2 * abs(slope0))  # Strong Wolfe

        # print(
        #     f"it={k:2d}, alpha={alpha:.3e}, phi={phi:.3e}, "
        #     f"phi'={phi_prime:.3e}, Armijo={armijo_ok}, Curv={curvature_ok}"
        # )

        # If both conditions satisfied → done
        if armijo_ok and curvature_ok:
            return float(alpha)

        # If Armijo fails → step too big → move right bound
        if not armijo_ok:
            alpha_R = alpha

        # If curvature fails but Armijo OK → step too small → move left bound
        elif not curvature_ok:
            alpha_L = alpha

        # New alpha by bisection
        alpha = 0.5 * (alpha_L + alpha_R)

        # Safety: if interval too small
        if abs(alpha_R - alpha_L) < 1e-12:
            print("Interval too small, stopping.")
            return float(alpha)

    print("Line search: max_iter reached, returning last alpha.")
    return float(alpha)

if __name__ == "__main__":
    base_x = np.array([2.0, 2.0])
    problem = rosenbrock()

    solution, iters, fmin = steepest_descent(
        x=base_x,
        problem=problem,
        tol=1e-6,
        iteration_limit=100000,
    )

    print("\nFINAL RESULTS:")
    print(f"Solution: {solution}")
    print(f"Iterations: {iters}")
    print(f"Minimum f(x): {fmin}")

"""
--------------------------------------------------------
with this approaach line search is only called one at beginning and then with the
alpha is used a constant inside steepest descent.

need to call alpha inside steepest descent and thne find optimal alpha at
every iteration
-------------------------------------------------------
if __name__ == "__main__":
    base_x = np.array([2.0,2.0])
    problem = rosenbrock()
    direction = -(problem.grad_objective(base_x))
    # static step size, later replaced by line search
    alpha = linesearch(base_x=base_x,direction=direction)
    '''as we know steepest descent function return three values,
    so making 3 variables and assigned the returned values to them'''
    solution, iters, fmin = steepest_descent(
        x=base_x,
        problem=problem,
        alpha=alpha
    )
    '''alpha = linesearch(base_x=base_x)
    solution = steepest_descent(x=base_x,alpha=alpha) this is wrong,
    variable is assigned as per return value fo the function'''
    print("\nFINAL RESULTS:")
    print(f"Solution:{solution}")
    print(f"Iterations:{iters}")
    print(f"Minimum f(x):{fmin}")
"""
